In [17]:
import torch
import torch.nn as nn

class PINN(nn.Module):
    def __init__(self, layers):
        super(PINN, self).__init__()
        self.net = nn.Sequential()
        for i in range(len(layers)-1):
            self.net.add_module(f"linear_{i}", nn.Linear(layers[i], layers[i+1]))
            if i < len(layers)-2:
                self.net.add_module(f"tanh_{i}", nn.Tanh())
        
    def forward(self, x, t):
        inputs = torch.cat([x, t], dim=1)
        return self.net(inputs)

ModuleNotFoundError: No module named 'torch'

In [8]:
def physics_loss(model, x, t, alpha=0.1):
    """Вычисление потерь на уравнении теплопроводности"""
    x.requires_grad_(True)
    t.requires_grad_(True)
    
    u = model(x, t)
    
    # Первые производные
    u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u),
                            create_graph=True)[0]
    u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u),
                            create_graph=True)[0]
    
    # Вторая производная по x
    u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x),
                             create_graph=True)[0]
    
    # Уравнение теплопроводности: u_t - alpha * u_xx = 0
    physics = u_t - alpha * u_xx
    return torch.mean(physics**2)

def boundary_loss(model):
    """Потери на граничных условиях"""
    # Левая граница: x=0, t∈[0,1]
    t_left = torch.rand(100, 1, requires_grad=True)
    x_left = torch.zeros_like(t_left)
    u_left = model(x_left, t_left)
    loss_left = torch.mean(u_left**2)  # u(0,t) = 0
    
    # Правая граница: x=1, t∈[0,1]
    t_right = torch.rand(100, 1, requires_grad=True)
    x_right = torch.ones_like(t_right)
    u_right = model(x_right, t_right)
    loss_right = torch.mean(u_right**2)  # u(1,t) = 0
    
    return loss_left + loss_right

def initial_loss(model):
    """Потери на начальном условии"""
    x_initial = torch.rand(100, 1, requires_grad=True)
    t_initial = torch.zeros_like(x_initial)
    u_pred = model(x_initial, t_initial)
    u_exact = torch.sin(torch.pi * x_initial)  # u(x,0) = sin(πx)
    
    return torch.mean((u_pred - u_exact)**2)

In [ ]:
def total_loss(model, x_collocation, t_collocation):
    """Общая функция потерь"""
    # Коллокационные точки (внутри области)
    loss_physics = physics_loss(model, x_collocation, t_collocation)
    
    # Граничные условия
    loss_bc = boundary_loss(model)
    
    # Начальное условие
    loss_ic = initial_loss(model)
    
    # Балансировка весов (можно настроить)
    return (loss_physics + 
            10.0 * loss_bc + 
            10.0 * loss_ic)

In [ ]:
def total_loss(model, x_collocation, t_collocation):
    """Общая функция потерь"""
    # Коллокационные точки (внутри области)
    loss_physics = physics_loss(model, x_collocation, t_collocation)
    
    # Граничные условия
    loss_bc = boundary_loss(model)
    
    # Начальное условие
    loss_ic = initial_loss(model)
    
    # Балансировка весов (можно настроить)
    return (loss_physics + 
            10.0 * loss_bc + 
            10.0 * loss_ic)

In [ ]:
for epoch in range(10000):
    optimizer.zero_grad()
    
    loss = total_loss(model, x_collocation, t_collocation)
    
    loss.backward()
    optimizer.step()
    
    if epoch % 1000 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.6f}")

In [10]:
import matplotlib.pyplot as plt
import numpy as np

# Создание сетки для визуализации
x_test = np.linspace(0, 1, 100)
t_test = np.linspace(0, 1, 100)
X, T = np.meshgrid(x_test, t_test)

# Предсказание модели
x_tensor = torch.tensor(X.flatten()[:, None], dtype=torch.float32)
t_tensor = torch.tensor(T.flatten()[:, None], dtype=torch.float32)
U_pred = model(x_tensor, t_tensor)
U_pred = U_pred.detach().numpy().reshape(X.shape)

# Визуализация
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.contourf(T, X, U_pred, levels=50)
plt.colorbar(label='Temperature u(x,t)')
plt.xlabel('Time t')
plt.ylabel('Position x')
plt.title('PINN Solution')

plt.subplot(1, 2, 2)
# Сравнение с аналитическим решением в несколько моментов времени
t_selected = [0.0, 0.2, 0.5, 1.0]
for t_val in t_selected:
    x_plot = torch.tensor(np.linspace(0, 1, 100)[:, None], dtype=torch.float32)
    t_plot = torch.ones_like(x_plot) * t_val
    u_plot = model(x_plot, t_plot).detach().numpy()
    plt.plot(x_plot, u_plot, label=f't = {t_val}')

plt.xlabel('Position x')
plt.ylabel('Temperature u(x)')
plt.legend()
plt.title('Temperature profiles at different times')
plt.tight_layout()
plt.show()

NameError: name 'torch' is not defined

In [ ]:
def analytical_solution(x, t, alpha=0.1):
    return np.sin(np.pi * x) * np.exp(-alpha * np.pi**2 * t)

# Вычисление ошибки
u_analytical = analytical_solution(X, T)
error = np.abs(U_pred - u_analytical)

print(f"Mean Absolute Error: {np.mean(error):.6f}")
print(f"Maximum Error: {np.max(error):.6f}")